In [1]:
import numpy as np
import pandas as pd
import pickle
import pymorphy2
import re
from sklearn.feature_extraction.text import TfidfVectorizer

morph = pymorphy2.MorphAnalyzer()
stops = open('russian.txt').read().split()

In [2]:
data = pd.read_csv('data.csv', sep=';')
vectorizer = TfidfVectorizer(max_features=4000)
tfidf = vectorizer.fit(data['plot'])
X = vectorizer.transform(data['plot']).toarray()
pickle.dump(tfidf, open('tfidf.pickle', 'wb'))
pickle.dump(X, open('tfidf_matrix.pickle', 'wb'))    # https://yadi.sk/d/tbhfd_gJY41ofg

def predict(query):
    query = ' '.join(list(morph.parse(word)[0].normal_form for word in re.findall("[\w]+", query)))
    query = ' '.join([word for word in query.split() if word not in stops])
    vec = vectorizer.transform([query]).toarray().reshape(4000, 1)
    data = pd.read_csv('data.csv', sep=';')
    simil = []
    for vector, film in zip(X, data['film']):
        simil.append((np.dot(vector, vec)/np.linalg.norm(vector)*np.linalg.norm(vec), film))
    simil.sort(reverse=True)
    return simil[:5]

query = ('Ким Ки Тхэк раньше работал водителем, но давно уже потерял работу'
         'и теперь влачит жалкое существование вместе со своей женой Чхун Сук'
         'и двумя детьми выпускного возраста в полуподвальной квартире. Семья'
         'пытается свести концы с концами, складывая из картонок коробки для пиццы.')
#query = 'Сериал про Шерлока Холмса'
predict(query)

[(array([0.30845389]), 'Всё будет хорошо (фильм, 1998)'),
 (array([0.28651224]), 'Паразиты (фильм)'),
 (array([0.26740791]), 'Волчье лето'),
 (array([0.25781118]), 'Ким Пять-с-плюсом (фильм)'),
 (array([0.25449611]), 'Таксист (фильм, 2017)')]